In [1]:
import networkx as nx
import DyNetSetV2 as dns2
import pandas as pd
import json
import numpy as np
nx.__version__

'3.3'

In [2]:
graphType = "CM"
for n in np.arange(100, 1100, 100):
    G = nx.read_graphml("./output/null_models/CMgraph_"+str(n)+".graphml")
    print('G: ',G, G.size('weight'), nx.is_weakly_connected(G))

G:  DiGraph with 100 nodes and 664 edges 178350.0 True
G:  DiGraph with 200 nodes and 1414 edges 239028.0 True
G:  DiGraph with 300 nodes and 3341 edges 677123.0 True
G:  DiGraph with 400 nodes and 4439 edges 727372.0 True
G:  DiGraph with 500 nodes and 5518 edges 1104682.0 True
G:  DiGraph with 600 nodes and 8036 edges 1585848.0 True
G:  DiGraph with 700 nodes and 9508 edges 1652309.0 True
G:  DiGraph with 800 nodes and 11912 edges 2025266.0 True
G:  DiGraph with 900 nodes and 14845 edges 3197999.0 True
G:  DiGraph with 1000 nodes and 16144 edges 3019306.0 True


In [3]:
graphType = "ER"
for p in [0.2, 0.3]:
    for n in np.arange(100, 1100, 100):
        G = nx.read_graphml("./output/null_models/"+graphType+"graph_"+str(int(p*10))+"_"+str(n)+".graphml")
        print('G: ',G, G.size('weight'), nx.is_weakly_connected(G))


G:  DiGraph with 100 nodes and 1903 edges 250122.0 True
G:  DiGraph with 200 nodes and 7923 edges 1388085.0 True
G:  DiGraph with 300 nodes and 18029 edges 3058407.0 True
G:  DiGraph with 400 nodes and 31755 edges 5877554.0 True
G:  DiGraph with 500 nodes and 50057 edges 8747755.0 True
G:  DiGraph with 600 nodes and 71725 edges 12344668.0 True
G:  DiGraph with 700 nodes and 97958 edges 17553071.0 True
G:  DiGraph with 800 nodes and 127695 edges 23868371.0 True
G:  DiGraph with 900 nodes and 161969 edges 29346645.0 True
G:  DiGraph with 1000 nodes and 199341 edges 36010997.0 True
G:  DiGraph with 100 nodes and 2967 edges 468620.0 True
G:  DiGraph with 200 nodes and 11993 edges 2425304.0 True
G:  DiGraph with 300 nodes and 26876 edges 4720577.0 True
G:  DiGraph with 400 nodes and 47647 edges 8921103.0 True
G:  DiGraph with 500 nodes and 74669 edges 13669684.0 True
G:  DiGraph with 600 nodes and 107333 edges 19338116.0 True
G:  DiGraph with 700 nodes and 146481 edges 26941391.0 True
G:  D

In [4]:
import time
graphType = "CM"
results_CM = []
time_series_synergy_CM = {str(k):[] for k in np.arange(100, 1100, 100)}
for n in np.arange(100, 1100, 100):

    G = nx.read_graphml("./output/null_models/CMgraph_"+str(n)+".graphml")
    print('G: ',G, G.size('weight'), nx.is_weakly_connected(G))

    # ----------------------- measurements for G ----------------------- #
    # ascendency_ratio, reserve_ratio
    ascendency, reserve, capacity = dns2.ena_systemic_metrics_log2(G)
    # gini_index
    in_degrees = [deg for _, deg in G.in_degree(weight='weight')]
    out_degrees = [deg for _, deg in G.out_degree(weight='weight')]
    gini_in = dns2.gini(in_degrees)
    gini_out = dns2.gini(out_degrees)
    # store
    results_CM.append(('G', None, None, ascendency/capacity, reserve/capacity, gini_in, gini_out, G.number_of_nodes(), G.number_of_edges(), G.size('weight')))

    # ----------------------- dynamic netting ----------------------- #
    # cycle cancelling
    start = time.time()
    sg, not_sampled_edges = dns2.dag_sampling(G)
    for e in not_sampled_edges:
        volume_before = sg.size('weight') + e[2]['weight']
        sg = dns2.cycle_cancelling(e, sg)
        volume_after = sg.size('weight')
        time_series_synergy_CM[str(n)].append({"source": e[0], "target":e[1], "synergy":1-volume_after/volume_before})
    tg_dynet = sg.copy()
    # synergy
    CNS = 1-(tg_dynet.size('weight')/G.size('weight'))
    # execution_time
    execTime = time.time() - start
    # ascendency_ratio, reserve_ratio
    ascendency, reserve, capacity = dns2.ena_systemic_metrics_log2(tg_dynet)
    # gini_index
    in_degrees = [deg for _, deg in tg_dynet.in_degree(weight='weight')]
    out_degrees = [deg for _, deg in tg_dynet.out_degree(weight='weight')]
    gini_in = dns2.gini(in_degrees)
    gini_out = dns2.gini(out_degrees)
    # store
    results_CM.append(('tg_dynet', execTime, CNS, ascendency/capacity, reserve/capacity, gini_in, gini_out, G.number_of_nodes(), G.number_of_edges(), G.size('weight')))

    # ----------------------- static netting ----------------------- #
    # min cost graph 
    start = time.time()
    tg = dns2.min_cost_graph_transformation(G)
    tg_statnet = tg.copy()
    # synergy
    CNS = 1-(tg_statnet.size('weight')/G.size('weight'))
    # execution_time
    execTime = time.time() - start
    # ascendency_ratio, reserve_ratio
    ascendency, reserve, capacity = dns2.ena_systemic_metrics_log2(tg_statnet)
    # gini_index
    in_degrees = [deg for _, deg in tg_statnet.in_degree(weight='weight')]
    out_degrees = [deg for _, deg in tg_statnet.out_degree(weight='weight')]
    gini_in = dns2.gini(in_degrees)
    gini_out = dns2.gini(out_degrees)
    # store
    results_CM.append(('tg_statnet', execTime, CNS, ascendency/capacity, reserve/capacity, gini_in, gini_out, G.number_of_nodes(), G.number_of_edges(), G.size('weight')))
    # Export JSON file
    with open('./output/CM_synergy_data_'+str(n)+'.json', 'w') as f:
        json.dump(time_series_synergy_CM[str(n)], f, indent=4)
# export CSV file
pd.DataFrame(results_CM, columns=["graph_type", "execu_time", "synergy", "asc_ratio", "rsr_ratio", "gini_in", "gini_out", "nodes", "edges", "volume"]).to_csv('./output/results_CM.csv')


G:  DiGraph with 100 nodes and 664 edges 178350.0 True
G:  DiGraph with 200 nodes and 1414 edges 239028.0 True
G:  DiGraph with 300 nodes and 3341 edges 677123.0 True
G:  DiGraph with 400 nodes and 4439 edges 727372.0 True
G:  DiGraph with 500 nodes and 5518 edges 1104682.0 True
G:  DiGraph with 600 nodes and 8036 edges 1585848.0 True
G:  DiGraph with 700 nodes and 9508 edges 1652309.0 True
G:  DiGraph with 800 nodes and 11912 edges 2025266.0 True
G:  DiGraph with 900 nodes and 14845 edges 3197999.0 True
G:  DiGraph with 1000 nodes and 16144 edges 3019306.0 True


In [5]:
import time
graphType = "ER"
results_ER = []
time_series_synergy_ER = {str(k):[] for k in np.arange(100, 1100, 100)}
for p in [0.2, 0.3]:
    print(str(p)+"/--------------------/"+str(p))
    for n in np.arange(100, 1100, 100):

        G = nx.read_graphml("./output/null_models/"+graphType+"graph_"+str(int(p*10))+"_"+str(n)+".graphml")
        print('G: ',G, G.size('weight'), nx.is_weakly_connected(G))

        # ----------------------- measurements for G ----------------------- #
        # ascendency_ratio, reserve_ratio
        ascendency, reserve, capacity = dns2.ena_systemic_metrics_log2(G)
        # gini_index
        in_degrees = [deg for _, deg in G.in_degree(weight='weight')]
        out_degrees = [deg for _, deg in G.out_degree(weight='weight')]
        gini_in = dns2.gini(in_degrees)
        gini_out = dns2.gini(out_degrees)
        # store
        results_ER.append(('G', None, None, ascendency/capacity, reserve/capacity, gini_in, gini_out, G.number_of_nodes(), G.number_of_edges(), G.size('weight')))

        # ----------------------- dynamic netting ----------------------- #
        # cycle cancelling
        start = time.time()
        sg, not_sampled_edges = dns2.dag_sampling(G)
        for e in not_sampled_edges:
            volume_before = sg.size('weight') + e[2]['weight']
            sg = dns2.cycle_cancelling(e, sg)
            volume_after = sg.size('weight')
            time_series_synergy_CM[str(n)].append({"source": e[0], "target":e[1], "synergy":1-volume_after/volume_before})
        tg_dynet = sg.copy()
        # synergy
        CNS = 1-(tg_dynet.size('weight')/G.size('weight'))
        # execution_time
        execTime = time.time() - start
        # ascendency_ratio, reserve_ratio
        ascendency, reserve, capacity = dns2.ena_systemic_metrics_log2(tg_dynet)
        # gini_index
        in_degrees = [deg for _, deg in tg_dynet.in_degree(weight='weight')]
        out_degrees = [deg for _, deg in tg_dynet.out_degree(weight='weight')]
        gini_in = dns2.gini(in_degrees)
        gini_out = dns2.gini(out_degrees)
        # store
        results_ER.append(('tg_dynet', execTime, CNS, ascendency/capacity, reserve/capacity, gini_in, gini_out, G.number_of_nodes(), G.number_of_edges(), G.size('weight')))

        # ----------------------- static netting ----------------------- #
        # min cost graph 
        start = time.time()
        tg = dns2.min_cost_graph_transformation(G)
        tg_statnet = tg.copy()
        # synergy
        CNS = 1-(tg_statnet.size('weight')/G.size('weight'))
        # execution_time
        execTime = time.time() - start
        # ascendency_ratio, reserve_ratio
        ascendency, reserve, capacity = dns2.ena_systemic_metrics_log2(tg_statnet)
        # gini_index
        in_degrees = [deg for _, deg in tg_statnet.in_degree(weight='weight')]
        out_degrees = [deg for _, deg in tg_statnet.out_degree(weight='weight')]
        gini_in = dns2.gini(in_degrees)
        gini_out = dns2.gini(out_degrees)
        # store
        results_ER.append(('tg_statnet', execTime, CNS, ascendency/capacity, reserve/capacity, gini_in, gini_out, G.number_of_nodes(), G.number_of_edges(), G.size('weight')))
        # Export JSON file
        with open('./output/ER_synergy_data_'+str(n)+'.json', 'w') as f:
            json.dump(time_series_synergy_CM[str(n)], f, indent=4)
        # print( time.time() - start )
#  export CSV
pd.DataFrame(results_ER, columns=["graph_type", "execu_time", "synergy", "asc_ratio", "rsr_ratio", "gini_in", "gini_out", "nodes", "edges", "volume"]).to_csv('./output/results_ER.csv')

0.2/--------------------/0.2
G:  DiGraph with 100 nodes and 1903 edges 250122.0 True
0.03873395919799805
G:  DiGraph with 200 nodes and 7923 edges 1388085.0 True
0.1871321201324463
G:  DiGraph with 300 nodes and 18029 edges 3058407.0 True
0.6034128665924072
G:  DiGraph with 400 nodes and 31755 edges 5877554.0 True
1.471299171447754
G:  DiGraph with 500 nodes and 50057 edges 8747755.0 True
2.629749059677124
G:  DiGraph with 600 nodes and 71725 edges 12344668.0 True
3.8776798248291016
G:  DiGraph with 700 nodes and 97958 edges 17553071.0 True
6.042001008987427
G:  DiGraph with 800 nodes and 127695 edges 23868371.0 True
9.92394495010376
G:  DiGraph with 900 nodes and 161969 edges 29346645.0 True
11.21303915977478
G:  DiGraph with 1000 nodes and 199341 edges 36010997.0 True
17.556270122528076
0.3/--------------------/0.3
G:  DiGraph with 100 nodes and 2967 edges 468620.0 True
0.06717491149902344
G:  DiGraph with 200 nodes and 11993 edges 2425304.0 True
0.359605073928833
G:  DiGraph with 30

In [6]:
pd.read_csv('./output/results_CM.csv')

,Unnamed: 0,graph_type,execu_time,synergy,asc_ratio,rsr_ratio,gini_in,gini_out,nodes,edges,volume
0,0,G,NaN,NaN,0.579761,0.420239,0.895234,0.891825,100,664,178350.0
1,1,tg_dynet,0.124507,0.022473,0.603712,0.396288,0.901206,0.899350,100,664,178350.0
2,2,tg_statnet,0.010897,0.007586,0.588115,0.411885,0.897862,0.895380,100,664,178350.0
3,3,G,NaN,NaN,0.460058,0.539942,0.872094,0.872909,200,1414,239028.0
4,4,tg_dynet,0.462499,0.040066,0.456291,0.543709,0.878961,0.872332,200,1414,239028.0
5,5,tg_statnet,0.024572,0.007702,0.463546,0.536454,0.875491,0.873650,200,1414,239028.0
6,6,G,NaN,NaN,0.489200,0.510800,0.877398,0.885126,300,3341,677123.0
7,7,tg_dynet,2.154614,0.095569,0.481050,0.518950,0.877802,0.882267,300,3341,677123.0
8,8,tg_statnet,0.087750,0.011445,0.497664,0.502336,0.880868,0.887451,300,3341,677123.0
9,9,G,NaN,NaN,0.461152,0.538848,0.870410,0.865512,400,4439,727372.0
